import libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re

read in data

In [11]:
nps = pd.read_csv("Georgia_NPs_AddressesNPIs_new.csv")
display(nps)

,NPI,Entity_Type_Code,Provider_Last_Name,Provider_First_Name,Provider_Credentials,Street1,Street2,City,State,ZIP,...,Provider_Enumeration_Date,Provider_Sex,Provider_Taxonomy_Code,Provider_License_Number_1,Healthcare_Provider_Primary_Tax,Certification_Date,MEDICARE_SPECIALTY_CODE,MEDICARE_PROVIDER_SUPPLIER_TYPE,PROVIDER_TAXONOMY_DESCRIPTION_,NP_Type
0,1003095977,1,NICHOLAS,TRACY,NP,1968 PEACHTREE RD NW,PIEDMONT HO,ATLANTA,GA,303091281,...,10/29/2007,F,363LA2100X,185022,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
1,1003104670,1,DODD,JOHN,NP,2501 N PATTERSON ST,NaN,VALDOSTA,GA,316021735,...,7/18/2011,M,363LA2100X,RN117801,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
2,1003128240,1,SARNO,JENNIFER,ACNP-BC,780 CANTON RD NE,SUITE 400,MARIETTA,GA,300607241,...,7/3/2010,F,363LA2100X,177164,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
3,1003222191,1,SAUCEDO,ISRAEL,"APRN, FN",417 W 3RD AVE,NaN,ALBANY,GA,317011943,...,7/9/2014,M,363LA2100X,RN170526,N,6/29/2022,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
4,1003280975,1,SHUMAKER-KIRK,CHARVETTE,NP-C,600 CELEBRATE LIFE PKWY,NaN,NEWNAN,GA,302658001,...,11/16/2015,F,363LA2100X,RN193221,Y,5/30/2023,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14462,1982088340,1,STONE,SARAH,APRN-CNP,3278 MITCHELL BLVD,NaN,MOODY AFB,GA,316995529,...,7/16/2015,F,363LW0102X,RN176410,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14463,1982210860,1,VINES,KRISTI,WHNP,10692 MEDLOCK BRIDGE RD,NaN,JOHNS CREEK,GA,300971890,...,9/16/2020,F,363LW0102X,RN255722,Y,9/16/2020,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14464,1982679189,1,KINNEY,KIMBERLY,"RN, NP",5354 REYNOLDS ST STE 422,NaN,SAVANNAH,GA,314056011,...,2/21/2006,F,363LW0102X,24164385,N,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14465,1992235469,1,CHAPMAN,NIJAH,WHNP,2665 N DECATUR RD STE 255,NaN,DECATUR,GA,300336176,...,6/15/2017,F,363LW0102X,RN228375,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP


In [15]:
# NP-type credentials you want to recognize and preserve as-is
KNOWN_CREDS = [
    # Primary Care & Family Focused
    "FNP", "AGPCNP", "PNP-PC", "WHNP", 
    # Acute & Hospital-Based Care
    "AGACNP", "PNP-AC", "ACNP", 
    # Population & Specialty Focused
    "PMHNP", "NNP", "ENP", "ONP", "A-GNP", "GNP", "ANP", "ONP-C", 
    # Less Common or Retired Certifications
    "CNP", "SNP", "AANP-BC", "ANCC-BC"
]

KNOWN_CREDS_SORTED = sorted(KNOWN_CREDS, key=len, reverse=True)
KNOWN_RE = re.compile(r"(?:%s)" % "|".join(map(re.escape, KNOWN_CREDS_SORTED)))

SYNONYMS = {
    "PH.D": "PHD", "PH D": "PHD", "P H D": "PHD",
    "D.N.P": "DNP", "M.S.N": "MSN", "B.S.N": "BSN", "R.N": "RN", "N.P": "NP",
    "phd": "PHD", "dnp": "DNP", "msn": "MSN", "bsn": "BSN", "rn": "RN", "np": "NP",
    "fnp": "FNP", "pmhnp": "PMHNP", "agnp": "AGNP", "pnp": "PNP", "whnp": "WHNP",
    "aprn": "APRN", "crna": "CRNA", "cnm": "CNM", "cns": "CNS", "cnl": "CNL",
    "md": "MD", "do": "DO", "pa": "PA", "pa-c": "PA-C",
}

def normalize_token(token: str) -> str:
    """Basic normalization of individual tokens."""
    t = token.strip()
    if not t:
        return ""
    t = t.replace(".", "")                 # drop periods
    t = re.sub(r"\s+", " ", t).strip()     # collapse spaces
    key = t.lower()
    if key in SYNONYMS:
        return SYNONYMS[key]
    return t.upper()

def split_stuck_credentials(token: str) -> list[str]:
    """
    Split tokens that are stuck together using KNOWN_CREDS,
    but DO NOT split things that already contain '-' or '/'.
    """
    s = normalize_token(token)
    if not s:
        return []
    # if it has '-' or '/', treat as one spec/credential (don't decompose)
    if "-" in s or "/" in s:
        return [s]
    if s in KNOWN_CREDS:
        return [s]
    # greedy match using known credentials
    out, i = [], 0
    while i < len(s):
        m = KNOWN_RE.match(s, i)
        if not m:
            return [s]  # can't decompose reliably; keep as-is
        out.append(m.group(0))
        i = m.end()
    return out

def cell_already_standard(x: str) -> bool:
    """
    Check if, after light normalization, ALL tokens in x are exactly in KNOWN_CREDS.
    If yes, we will leave the cell unchanged.
    """
    s = x.strip()
    s = s.replace(".", "")                 # ignore periods
    s = re.sub(r"\s*,\s*", ",", s)         # normalize comma spacing
    tokens = [t.strip().upper() for t in s.split(",") if t.strip()]
    if not tokens:
        return False
    return all(t in KNOWN_CREDS for t in tokens)

def clean_credentials_cell(x):
    """
    Clean Provider_Credentials cell:
    - If it already consists only of credentials in KNOWN_CREDS → leave as-is.
    - Else: normalize separators, keep '-' and '/', remove periods,
      split stuck tokens using KNOWN_CREDS, normalize, dedupe, sort in-cell.
    """
    if pd.isna(x) or not isinstance(x, str) or x.strip() == "":
        return x  # leave blanks/NaNs untouched

    # ✅ EARLY EXIT: keep existing cell as-is if all tokens are in KNOWN_CREDS
    if cell_already_standard(x):
        return x.strip()

    s = x.strip()

    # normalize list separators -> commas (keep '-' and '/')
    s = re.sub(r"[;|&]+", ",", s)
    s = re.sub(r"\band\b", ",", s, flags=re.IGNORECASE)

    # normalize spacing around hyphens and slashes instead of splitting on them
    s = re.sub(r"\s*-\s*", "-", s)
    s = re.sub(r"\s*/\s*", "/", s)

    # remove dangling leading/trailing hyphens (rare garbage)
    s = re.sub(r"^-+|-+$", "", s)

    # remove periods globally
    s = s.replace(".", "")

    # collapse whitespace and tidy commas
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\s*,\s*", ",", s)
    s = re.sub(r",+", ",", s).strip(",")

    if not s:
        return ""

    rough_tokens = [t for t in s.split(",") if t.strip()]

    exploded = []
    for t in rough_tokens:
        exploded.extend(split_stuck_credentials(t))

    normalized = [normalize_token(t) for t in exploded if t]
    unique = sorted(set([t for t in normalized if t]))

    return ",".join(unique)

# Apply to your column
nps['Provider_Credentials'] = nps['Provider_Credentials'].apply(clean_credentials_cell)

# sort rows by the cleaned column (optional)
nps = nps.sort_values(by='Provider_Credentials', ascending=True).reset_index(drop=True)

display(nps)


,NPI,Entity_Type_Code,Provider_Last_Name,Provider_First_Name,Provider_Credentials,Street1,Street2,City,State,ZIP,...,Provider_Enumeration_Date,Provider_Sex,Provider_Taxonomy_Code,Provider_License_Number_1,Healthcare_Provider_Primary_Tax,Certification_Date,MEDICARE_SPECIALTY_CODE,MEDICARE_PROVIDER_SUPPLIER_TYPE,PROVIDER_TAXONOMY_DESCRIPTION_,NP_Type
0,1932582301,1,JIMENEZ,SARAH,"A,APN,C",1120 15TH ST,NaN,AUGUSTA,GA,30912,...,7/3/2015,F,363L00000X,26NJ00573000,N,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,"NP, No Subspecialty Noted"
1,1720435514,1,CHEWNING,GARY,"A,C,FNP",655 JESSE JEWELL PKWY SE STE B,NaN,GAINESVILLE,GA,305013854,...,5/21/2016,M,363LF0000X,RN208793,N,1/30/2024,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Family NP
2,1548556426,1,MILEY,BARBARA,"A,C,FNP",3000 HOSPITAL BLVD,NaN,ROSWELL,GA,300764915,...,6/20/2011,F,363LA2100X,RN165608,N,3/28/2025,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Acute Care NP
3,1851475107,1,PALERINO,LESA,"A,C,FNP",7141 MOON RD,SUITE B,COLUMBUS,GA,319097294,...,10/24/2006,F,363LF0000X,RN077219 NP,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Family NP
4,1720508567,1,FLOYD,CHRISTOPH,"A,C,FNP",1641 MADISON AVE,NaN,TIFTON,GA,317943757,...,6/22/2017,M,363LF0000X,213107,Y,3/23/2021,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Family NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14462,1902220429,1,PETITO,ANDI,NaN,3110 CLIFTON SPRINGS RD,NaN,DECATUR,GA,300344600,...,2/5/2014,F,363LW0102X,RN225020,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14463,1932484201,1,WATERS,CATHLEEN,NaN,604 RIVERSIDE AVE,NaN,WAYCROSS,GA,315015323,...,10/18/2011,F,363LW0102X,RN167697,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14464,1952026031,1,CENTER,SYDNEY,NaN,1595 KENNESAW DUE WEST RD NW STE 100,NaN,KENNESAW,GA,301527640,...,10/11/2022,F,363LW0102X,RN309563,Y,12/7/2023,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP
14465,1952876369,1,WELLS,JESSICA,NaN,477 WINDSOR ST SW,NaN,ATLANTA,GA,303122530,...,10/11/2018,F,363LW0102X,RN185551,Y,NaN,50,Nurse Practitioner,Physician Assistants & Advanced Practice Nursi...,Womens Health NP


In [16]:
# Create a frequency table of Provider_Credentials
credential_counts = (
    nps['Provider_Credentials']
    .value_counts()
    .rename_axis('Provider_Credentials')
    .reset_index(name='Count')
    .sort_values(by='Provider_Credentials', ascending=True)
    .reset_index(drop=True)
)

# Display the summary table
display(credential_counts)

,Provider_Credentials,Count
0,"A,APN,C",1
1,"A,C,FNP",6
2,"A,C,GNP",7
3,"A,CPNP",1
4,"A,G,MSN",1
...,...,...
468,W.H.N.P.,1
469,WHCNP,2
470,WHNP,64
471,WHPN,1


In [17]:
mask_unspecified = nps["NP_Type"].str.contains("no subspecialty", case=False, na=False)
nps.loc[mask_unspecified, "NP_Type"].value_counts()

nps.loc[mask_unspecified, "Provider_Credentials"].value_counts()

Provider_Credentials
NP         837
C,NP       320
C,FNP      202
FNP        153
APRN        76
          ... 
C,FNP,G      1
ANP,CNS      1
C,E,FNP      1
C,DN,NP      1
A,APN,C      1
Name: count, Length: 156, dtype: int64